In [1]:
import os
os.chdir(r"C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification")
print("Current directory:", os.getcwd())

Current directory: C:\Users\user\Python Programs\Resume Projects\End-to-End-Chicken-Disease-Classification


In [2]:
from dataclasses import dataclass
from pathlib import Path

In [3]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weight: str
    params_classes:int

In [4]:
from cnnProject.constants import CONFIG_FILE_PATH,PARAMS_FILE_PATH
from cnnProject.utils.common import readYamlFile,createDirectories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=readYamlFile(config_file_path)
        self.params=readYamlFile(params_file_path)
        createDirectories([self.config.artifacts_root])

    def get_prepare_base_model_config(self)->PrepareBaseModelConfig:
        config=self.config.prepare_base_model
        createDirectories([config.root_dir])

        prepare_base_model_config=PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_classes=self.params.CLASSES,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS
        )

        return prepare_base_model_config

In [24]:
import tensorflow as tf

In [25]:
class PrepareBaseModel:
    def __init__(self,config:PrepareBaseModelConfig):
        self.config=config
    
    def get_base_model(self):
        self.model=tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top
        )
        self.saveModel(path=self.config.base_model_path,model=self.model)
    
    @staticmethod
    def _prepare_full_mode(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable=False
        elif(freeze_till is not None) and (freeze_till>0):
            for layer in model.layers[:-freeze_till]:
                model.trainable=False
        
        flatten_in=tf.keras.layers.Flatten()(model.output)
        prediction=tf.keras.layers.Dense(units=classes,activation="softmax")(flatten_in)
        full_model=tf.keras.models.Model(inputs=model.input,outputs=prediction)

        full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),loss=tf.keras.losses.CategoricalCrossentropy(),metrics=["accuracy"])

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model=self._prepare_full_mode(model=self.model,classes=self.config.params_classes,freeze_all=True,freeze_till=None,learning_rate=self.config.params_learning_rate)
        self.saveModel(path=self.config.updated_base_model_path,model=self.full_model)


    @staticmethod
    def saveModel(path:Path,model=tf.keras.Model):
        model.save(path)

In [26]:
try:
    config=ConfigurationManager()
    prepare_base_model_config=config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2025-06-28 20:48:00,023 : INFO : common : the yaml file form the path config\config.yaml loaded Sucessfully]
[2025-06-28 20:48:00,025 : INFO : common : the yaml file form the path params.yaml loaded Sucessfully]
[2025-06-28 20:48:00,026 : INFO : common : Directories created Sucessfully at artifacts]
[2025-06-28 20:48:00,027 : INFO : common : Directories created Sucessfully at artifacts/prepare_base_model]
[2025-06-28 20:48:00,245 : WARNING : saving_utils : Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                       